In [1]:
!pip install numpy pandas scikit-learn tensorflow deap


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 1.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Content/KDDTrain+.txt', header=None)

# Define feature columns and target column
feature_columns = data.columns[:-1]
target_column = data.columns[-1]

# Identify columns with non-numerical data
non_numeric_columns = data[feature_columns].select_dtypes(include=['object']).columns

# Encode non-numerical features using LabelEncoder
for column in non_numeric_columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])

# Preprocess the features
scaler = MinMaxScaler()
X = scaler.fit_transform(data[feature_columns])

# Encode the target variable
y = pd.get_dummies(data[target_column])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Mounted at /content/drive


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def build_model(input_dim, output_dim):
    model = Sequential([
        Dense(64, input_dim=input_dim, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(output_dim, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_model(X_train, y_train):
    model = build_model(X_train.shape[1], y_train.shape[1])
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
    return model

def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy


In [4]:
import random
from deap import base, creator, tools, algorithms

# Define the fitness function
def evaluate(individual):
    selected_features = [index for index, value in enumerate(individual) if value == 1]
    if len(selected_features) == 0:
        return 0,

    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    model = train_model(X_train_selected, y_train)
    accuracy = evaluate_model(model, X_test_selected, y_test)
    return accuracy,

# Define GA components
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(feature_columns))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Run the GA
def run_ga():
    population = toolbox.population(n=50)
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, verbose=True)
    return tools.selBest(population, k=1)[0]

best_individual = run_ga()
selected_features = [index for index, value in enumerate(best_individual) if value == 1]


gen	nevals
0  	50    
1  	38    
2  	33    
3  	28    
4  	32    
5  	34    
6  	35    
7  	25    
8  	27    
9  	26    
10 	32    


In [5]:
# Train the model using the selected features
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]

print("Training model with selected features...")
model = build_model(X_train_selected.shape[1], y_train.shape[1])
model.fit(X_train_selected, y_train, epochs=2000, batch_size=32, validation_data=(X_test_selected, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_selected, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Training model with selected features...
Epoch 1/2000
270/270 [==============================] - 2s 3ms/step - loss: 1.6610 - accuracy: 0.4797 - val_loss: 1.2867 - val_accuracy: 0.5355
Epoch 2/2000
270/270 [==============================] - 1s 3ms/step - loss: 1.3641 - accuracy: 0.5109 - val_loss: 1.2009 - val_accuracy: 0.6064
Epoch 3/2000
270/270 [==============================] - 1s 3ms/step - loss: 1.2752 - accuracy: 0.5369 - val_loss: 1.1526 - val_accuracy: 0.6273
Epoch 4/2000
270/270 [==============================] - 1s 5ms/step - loss: 1.2331 - accuracy: 0.5523 - val_loss: 1.1187 - val_accuracy: 0.6365
Epoch 5/2000
270/270 [==============================] - 2s 7ms/step - loss: 1.1868 - accuracy: 0.5706 - val_loss: 1.0873 - val_accuracy: 0.6361
Epoch 6/2000
270/270 [==============================] - 2s 9ms/step - loss: 1.1658 - accuracy: 0.5795 - val_loss: 1.0675 - val_accuracy: 0.6236
Epoch 7/2000
270/270 [==============================] - 2s 7ms/step - loss: 1.1401 - accuracy: 